### Optimization methods: random and hillclimber

In [1]:
import pandas as pd
import random
import copy

In [2]:
import geopandas as gpd
import shapely

source = gpd.read_file('../data/Inzameling_huisvuil_100220.shp')
source = source[source['aanbiedwij'] == 'Breng uw restafval  naar een container voor restafval.']
polygons = list(source.geometry)

def adress_in_service_area(x, y, polygon_list):
    """
    function to see whether a certain household is within the service area of rest
    Input is x and y coordinates of a house and a list of polygons of service area.
    Returns boolean
    """
    point = shapely.geometry.Point(float(x),float(y))
    for polygon in polygon_list:
        if polygon.contains(point):
            return True
    return False

In [3]:
def calculate_weighted_distance(good_result):
    rest_mean = good_result['rest_afstand'].mean()
    papier_mean = good_result['papier_afstand'].mean()
    glas_mean = good_result['glas_afstand'].mean()
    plastic_mean = good_result['plastic_afstand'].mean()
    textiel_mean = good_result['textiel_afstand'].mean()
#     print(rest_mean, papier_mean, glas_mean, plastic_mean, textiel_mean)
    score = 0.35 * rest_mean + 0.25 * plastic_mean + 0.2 * papier_mean + 0.15 * glas_mean + 0.05 * textiel_mean
    return score

In [4]:
cluster_join = pd.read_csv('../data/cluster_join.csv', compression='gzip', index_col=0)

C:\Users\corne\anaconda3\envs\geo_env\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
all_households1 = cluster_join[['naar_s1_afv_nodes', 'bk_afv_rel_nodes_poi']].drop_duplicates()
all_households1['woning'] = all_households1['bk_afv_rel_nodes_poi'].str.split('~')
all_households1['woning_x'] = all_households1['woning'].apply(lambda x: x[0])
all_households1['woning_y'] = all_households1['woning'].apply(lambda x: x[1])
all_households1 = all_households1.drop('bk_afv_rel_nodes_poi', axis=1)

In [6]:
all_households1['uses_container'] = all_households1.apply(lambda row: adress_in_service_area(row['woning_x'], row['woning_y'], polygons), axis=1)
good_result = all_households1[all_households1['uses_container']]

In [7]:
def random_shuffling_clusters(cluster_join = cluster_join):
    df = cluster_join.groupby('van_s1_afv_nodes').first()[['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']]
    df = df.fillna(0)
    
    fractionlist = df['rest'].astype('int').sum() * ['rest'] + df['plastic']\
    .astype('int').sum() * ['plastic'] + df['papier'].astype('int').sum() * \
    ['papier'] + df['glas'].astype('int').sum() * ['glas'] + df['textiel'].\
    astype('int').sum() * ['textiel']
    random.shuffle(fractionlist)
    
    cluster_list = []
    
    for i in df.index:
        cluster_list.extend([str(i)] * df.loc[i].totaal.astype('int'))
    
    df_new = pd.DataFrame([cluster_list, fractionlist]).T.rename(columns={0:'poi', 1:'fractie'})
    df_new_apply = df_new.groupby('poi').fractie.value_counts().unstack()
    
    cluster_join1 = cluster_join.drop(['rest', 'plastic', 'papier', 'glas', 'textiel'], axis=1)
    cluster_join1['van_s1_afv_nodes'] = cluster_join1['van_s1_afv_nodes'].astype('int')
    
    df_new_apply.index = df_new_apply.index.astype('int')
    cluster_join1 = cluster_join1.set_index('van_s1_afv_nodes')
    return cluster_join1.join(df_new_apply, how='left').reset_index()\
            .rename(columns={'index': 'van_s1_afv_nodes'})\
            .sort_values('afstand')

In [8]:
shuffled_join = random_shuffling_clusters()

In [11]:
shortest_rest = shuffled_join[shuffled_join['rest'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_rest', 'afstand': 'rest_afstand'})
shortest_plastic = shuffled_join[shuffled_join['plastic'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_plastic', 'afstand': 'plastic_afstand'})
shortest_papier = shuffled_join[shuffled_join['papier'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_papier', 'afstand': 'papier_afstand'})
shortest_glas = shuffled_join[shuffled_join['glas'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_glas', 'afstand': 'glas_afstand'})
shortest_textiel = shuffled_join[shuffled_join['textiel'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_textiel', 'afstand': 'textiel_afstand'})

all_households = all_households1.set_index('naar_s1_afv_nodes').join([shortest_rest, shortest_plastic, shortest_papier, shortest_glas, shortest_textiel], how='left')
good_result = all_households[all_households['uses_container']]

In [12]:
def create_aanlsuitingen(good_result, total_join):
    """
    Function that returns dataframe aansluitingen that calculates amount of
    households per cluster and the percentage
    """
    aansluitingen = pd.DataFrame(good_result['poi_rest'].value_counts()).\
                join(pd.DataFrame(good_result['poi_papier'].value_counts()), how='outer').\
                join(pd.DataFrame(good_result['poi_plastic'].value_counts()), how='outer').\
                join(pd.DataFrame(good_result['poi_glas'].value_counts()), how='outer').\
                join(pd.DataFrame(good_result['poi_textiel'].value_counts()), how='outer')
    
    tmp_for_join = total_join[['van_s1_afv_nodes', 'rest', 'papier', 'plastic', 'glas', 'textiel', 'totaal']].drop_duplicates().set_index('van_s1_afv_nodes')
    aansluitingen = aansluitingen.join(tmp_for_join, how='left')
    
    aansluitingen['rest_perc'] = aansluitingen['poi_rest'] / aansluitingen['rest']
    aansluitingen['plastic_perc'] = aansluitingen['poi_plastic'] / aansluitingen['plastic'] / 2
    aansluitingen['papier_perc'] = aansluitingen['poi_papier'] / aansluitingen['papier'] / 2
    aansluitingen['glas_perc'] = aansluitingen['poi_glas'] / aansluitingen['glas'] / 2
    aansluitingen['textiel_perc'] = aansluitingen['poi_textiel'] / aansluitingen['textiel'] / 7.5

    return aansluitingen

aansluitingen = create_aanlsuitingen(good_result, shuffled_join)

In [20]:
def calculate_penalties(good_result, aansluitingen=aansluitingen):
    penalty1 = good_result[good_result['rest_afstand'] > 100]
    penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1.shape[0])/good_result.shape[0] * 0.35
    penalty2 = good_result[good_result['plastic_afstand'] > 150]
    penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2.shape[0])/good_result.shape[0] * 0.25
    penalty3 = good_result[good_result['papier_afstand'] > 150]
    penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3.shape[0])/good_result.shape[0] * 0.2
    penalty4 = good_result[good_result['glas_afstand'] > 150]
    penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4.shape[0])/good_result.shape[0] * 0.15
    penalty5 = good_result[good_result['textiel_afstand'] > 300]
    penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5.shape[0])/good_result.shape[0] * 0.05

    penalty6 = aansluitingen[aansluitingen['rest_perc'] > 100]
    penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result.shape[0] * 0.35 * 1000
    penalty7 = aansluitingen[aansluitingen['plastic_perc'] > 100]
    penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result.shape[0] * 0.25 * 1000
    penalty8 = aansluitingen[aansluitingen['papier_perc'] > 100]
    penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result.shape[0] * 0.2 * 1000
    penalty9 = aansluitingen[aansluitingen['glas_perc'] > 100]
    penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result.shape[0] * 0.15 * 1000
    penalty10 = aansluitingen[aansluitingen['textiel_perc'] > 100]
    penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result.shape[0] * 0.05 * 1000

    total_penalties = sum([penalty1_sum, penalty2_sum, penalty3_sum, penalty4_sum, penalty5_sum,\
                           penalty6_sum, penalty7_sum, penalty8_sum, penalty9_sum, penalty10_sum])
    return total_penalties

def calculate_simple_penalties(good_result, aansluitingen=aansluitingen):
    penalty1 = good_result[good_result['rest_afstand'] > 100].shape[0]
    penalty2 = good_result[good_result['plastic_afstand'] > 150].shape[0]
    penalty3 = good_result[good_result['papier_afstand'] > 150].shape[0]
    penalty4 = good_result[good_result['glas_afstand'] > 150].shape[0]
    penalty5 = good_result[good_result['textiel_afstand'] > 300].shape[0]
    
    temp = (aansluitingen['poi_rest'] - aansluitingen['rest'] * 100)
    penalty6 = temp[temp > 0].sum()
    temp = (aansluitingen['poi_plastic'] - aansluitingen['plastic'] * 200)
    penalty7 = temp[temp > 0].sum()
    temp = (aansluitingen['poi_papier'] - aansluitingen['papier'] * 200)
    penalty8 = temp[temp > 0].sum()
    temp = (aansluitingen['poi_glas'] - aansluitingen['glas'] * 200)
    penalty9 = temp[temp > 0].sum()
    temp = (aansluitingen['poi_textiel'] - aansluitingen['textiel'] * 750)
    penalty10 = temp[temp > 0].sum()
    
    print(penalty1, penalty2, penalty3, penalty4, penalty5, penalty6, penalty7, penalty8, penalty9, penalty10)
    return penalty1+penalty2+penalty3+penalty4+penalty5+penalty6+penalty7+penalty8+penalty9+penalty10

### Calculate shortest distances and total score
Shortest distances are calculated, but also filtered on the right geolocations

In [14]:
avg_distance = calculate_weighted_distance(good_result)
penalties = calculate_penalties(good_result)
print(avg_distance, penalties, avg_distance+penalties)

178.88462666192294 116.88676862000099 295.7713952819239


## Repeated Random algorithm
Function to repeatedly perform the search for outcomes

In [15]:
def total_random():
    shuffled_join = random_shuffling_clusters()

    
    shortest_rest = shuffled_join[shuffled_join['rest'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_rest', 'afstand': 'rest_afstand'})
    shortest_plastic = shuffled_join[shuffled_join['plastic'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_plastic', 'afstand': 'plastic_afstand'})
    shortest_papier = shuffled_join[shuffled_join['papier'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_papier', 'afstand': 'papier_afstand'})
    shortest_glas = shuffled_join[shuffled_join['glas'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_glas', 'afstand': 'glas_afstand'})
    shortest_textiel = shuffled_join[shuffled_join['textiel'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_textiel', 'afstand': 'textiel_afstand'})

    all_households = all_households1.set_index('naar_s1_afv_nodes').join([shortest_rest, shortest_plastic, shortest_papier, shortest_glas, shortest_textiel], how='left')
    good_result = all_households[all_households['uses_container']]
    
    
    avg_distance = calculate_weighted_distance(good_result)
    aansluitingen = create_aanlsuitingen(good_result, shuffled_join)
    penalties = calculate_penalties(good_result)
    return avg_distance, penalties, avg_distance+penalties

In [ ]:
best = 9999
results = {}
for i in range(500):
    results[i] = total_random()
    if results[i][2] < best:
        best = results[i][2]
        print(best)
        print(results[i])

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient='index').rename(columns={0:'Average Distance', 1:'Penalties', 2:'Score'})
results_df.to_csv('random.csv')

In [ ]:
results_df['Score'].plot(title='Results of 500 random runs')

### Repeated hillclimber algorithm
Next up is to redefine an hillclimber algorithm that is able to repeatedly make small adjustments to the input candidate solution 

In [16]:
s = cluster_join
s = s.groupby('van_s1_afv_nodes').first()[['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].reset_index().rename(columns={'index': 'van_s1_afv_nodes'})
s = s.fillna(0)
best = 301.02

In [23]:
# Calculate score on hillclimber
def hillclimber_score(hillclimber_join, complicated=True):
    hill_shortest_rest = hillclimber_join[hillclimber_join['rest'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_rest', 'afstand': 'rest_afstand'})
    hill_shortest_plastic = hillclimber_join[hillclimber_join['plastic'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_plastic', 'afstand': 'plastic_afstand'})
    hill_shortest_papier = hillclimber_join[hillclimber_join['papier'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_papier', 'afstand': 'papier_afstand'})
    hill_shortest_glas = hillclimber_join[hillclimber_join['glas'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_glas', 'afstand': 'glas_afstand'})
    hill_shortest_textiel = hillclimber_join[hillclimber_join['textiel'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_textiel', 'afstand': 'textiel_afstand'})

    all_households = all_households1.set_index('naar_s1_afv_nodes').join([hill_shortest_rest, hill_shortest_plastic, hill_shortest_papier, hill_shortest_glas, hill_shortest_textiel], how='left')
    good_result = all_households[all_households['uses_container']]

    avg_distance = calculate_weighted_distance(good_result)
    aansluitingen = create_aanlsuitingen(good_result, hillclimber_join)
    if complicated:
        penalties = calculate_penalties(good_result)
    else:
        penalties = calculate_simple_penalties(good_result)
    return avg_distance, penalties, (avg_distance+penalties)

def initial_score(s, complicated=True):
    r= copy.deepcopy(s)
    
    cluster_join1 = cluster_join.drop(['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal'], axis=1)
    cluster_join1['van_s1_afv_nodes'] = cluster_join1['van_s1_afv_nodes'].astype('int')

    cluster_join1 = cluster_join1.set_index('van_s1_afv_nodes')
    r = r.set_index('van_s1_afv_nodes')
    hillclimber_join = cluster_join1.join(r, how='left').reset_index().sort_values(by='afstand', ascending=True)
    
    distance, penalties, score = hillclimber_score(hillclimber_join, complicated= complicated)
    
    return distance, penalties, score

In [24]:
def hillclimber(s, num_iterations, cluster_join = cluster_join, mod_max = 5, parameter='score', complicated=True):

    hillclimber_dict = {}
    distance, penalties, score = initial_score(s, complicated)
    hillclimber_dict[0] = [distance, penalties, score]
    
    if parameter == 'score':
        best = score
    if parameter == 'penalties':
        best = penalties
    
    for i in range(1, num_iterations+1):
        r = copy.deepcopy(s)
        fractions = ['rest', 'plastic', 'papier', 'glas', 'textiel']
        no_modifications = random.randint(1, mod_max)
#         print(no_modifications)
        for j in range(no_modifications):
            valid = False
            while not valid:
                location_a = random.randint(0, r.shape[0]-1)
                fraction_a = random.choice(fractions)
                location_b = random.randint(0, r.shape[0]-1)
                fraction_b = random.choice(fractions)

                if r.at[location_a, fraction_b] > 0 and r.at[location_b, fraction_a] > 0\
                                                    and fraction_a != fraction_b:
                    r.at[location_a, fraction_a] = int(r.at[location_a, fraction_a]) + 1
                    r.at[location_a, fraction_b] = int(r.at[location_a, fraction_b]) - 1
                    r.at[location_b, fraction_a] = int(r.at[location_a, fraction_b]) - 1
                    r.at[location_b, fraction_b] = int(r.at[location_a, fraction_b]) + 1
                    valid = True
    #                 print('succes')
#                     print(fraction_a, fraction_b)
    #             else:
    #                 print('failure')
    
        cluster_join1 = cluster_join.drop(['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal'], axis=1)
        cluster_join1['van_s1_afv_nodes'] = cluster_join1['van_s1_afv_nodes'].astype('int')

        cluster_join1 = cluster_join1.set_index('van_s1_afv_nodes')
        r = r.set_index('van_s1_afv_nodes')
        hillclimber_join = cluster_join1.join(r, how='left').reset_index().sort_values(by='afstand', ascending=True)
        
        distance, penalties, score = hillclimber_score(hillclimber_join, complicated)
        hillclimber_dict[i] = [distance, penalties, best, no_modifications]
        if parameter == 'score':
            print(score, best)
            if score < best:
                best = score
                hillclimber_join = s
        if parameter == 'penalties':
            print(penalties, best)
            if penalties < best:
                best = penalties
                hillclimber_join = s
    return hillclimber_dict, hillclimber_join

In [25]:
dct, best_config = hillclimber(s, 10, mod_max = 3, parameter='penalties', complicated=False)

27545 66548 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
27548 66553 43887 57068 51981 1002.0 11596.0 3957.0 5740.0 4081.0
273413.0 273361.0
27545 66551 43840 57179 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273475.0 273361.0
27556 66548 43840 57023 51922 1002.0 11596.0 3957.0 5740.0 4081.0
273265.0 273361.0
27546 66548 43851 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273373.0 273265.0
27559 66517 43855 57034 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273325.0 273265.0
27552 66548 43866 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273394.0 273265.0
27566 66548 43839 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273381.0 273265.0
27569 66548 43825 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273370.0 273265.0
27579 66537 43935 57118 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273529.0 273265.0
27545 66558 43864 57118 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273445.0 273265.0


### Hillclimber with random start

In [26]:
def best_of_random(i):
    best_configuration = copy.deepcopy(cluster_join)
    distance, penalties, score = initial_score(best_configuration.groupby('van_s1_afv_nodes').first()[['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].reset_index().rename(columns={'index': 'van_s1_afv_nodes'}))

    for j in range(i):
        shuffled_join = random_shuffling_clusters()    

        shortest_rest = shuffled_join[shuffled_join['rest'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_rest', 'afstand': 'rest_afstand'})
        shortest_plastic = shuffled_join[shuffled_join['plastic'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_plastic', 'afstand': 'plastic_afstand'})
        shortest_papier = shuffled_join[shuffled_join['papier'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_papier', 'afstand': 'papier_afstand'})
        shortest_glas = shuffled_join[shuffled_join['glas'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_glas', 'afstand': 'glas_afstand'})
        shortest_textiel = shuffled_join[shuffled_join['textiel'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_textiel', 'afstand': 'textiel_afstand'})

        all_households = all_households1.set_index('naar_s1_afv_nodes').join([shortest_rest, shortest_plastic, shortest_papier, shortest_glas, shortest_textiel], how='left')
        good_result = all_households[all_households['uses_container']]


        new_distance  = calculate_weighted_distance(good_result)
        aansluitingen = create_aanlsuitingen(good_result, shuffled_join)
        new_penalties = calculate_penalties(good_result)
        new_score = new_distance + new_penalties
        if new_score < score:
            score = new_score
            best_configuration = shuffled_join
    return best_configuration

In [ ]:
chang = best_of_random(25)
chang = chang.groupby('van_s1_afv_nodes').first()[['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].reset_index().rename(columns={'index': 'van_s1_afv_nodes'})
chang = chang.fillna(0)
dct = hillclimber(chang, 1000, mod_max = 3, parameter='penalties', complicated=False)


27545 66548 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
27545 66587 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273400.0 273361.0
27550 66548 43840 57040 52029 1002.0 11596.0 3957.0 5740.0 4081.0
273383.0 273361.0
27588 66548 43880 57079 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273455.0 273361.0
27548 66548 43840 57084 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273380.0 273361.0
27545 66548 43837 57116 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273406.0 273361.0
27537 66597 43855 57068 51885 1002.0 11596.0 3957.0 5740.0 4081.0
273318.0 273361.0
27553 66548 43795 57068 51951 1002.0 11596.0 3957.0 5740.0 4081.0
273291.0 273318.0
27545 66553 43840 57078 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273376.0 273291.0
27547 66543 43856 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273374.0 273291.0
27545 66548 43848 57052 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273353.0 273291.0
27545 66489 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273302.0 273291.0
27595 6654

27591 66548 43796 57074 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273369.0 273250.0
27566 66548 43840 57046 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273360.0 273250.0
27568 66565 43840 57084 52120 1002.0 11596.0 3957.0 5740.0 4081.0
273553.0 273250.0
27552 66531 43840 57078 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273361.0 273250.0
27562 66545 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273375.0 273250.0
27605 66548 43792 57020 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273325.0 273250.0
27562 66548 43840 57068 51973 1002.0 11596.0 3957.0 5740.0 4081.0
273367.0 273250.0
27587 66548 43869 57068 51929 1002.0 11596.0 3957.0 5740.0 4081.0
273377.0 273250.0
27574 66548 43853 57112 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273447.0 273250.0
27542 66548 43861 57150 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273461.0 273250.0
27548 66548 43870 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273394.0 273250.0
27576 66548 43850 57010 51938 1002.0 11596.0 3957.0 5740.0 4081.0
273298.0 2

27566 66519 43840 57236 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273521.0 272727.0
27548 66548 43853 57114 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273423.0 272727.0
27558 66548 43840 57068 51964 1002.0 11596.0 3957.0 5740.0 4081.0
273354.0 272727.0
27575 66548 43857 57103 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273443.0 272727.0
27551 66548 43907 57102 51980 1002.0 11596.0 3957.0 5740.0 4081.0
273464.0 272727.0
27547 66548 43879 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273402.0 272727.0
27545 66548 43878 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273399.0 272727.0
27545 66654 43849 57006 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273414.0 272727.0
27569 66489 43856 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273342.0 272727.0
27554 66548 43840 57100 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273402.0 272727.0
27556 66560 43831 57134 51694 1002.0 11596.0 3957.0 5740.0 4081.0
273151.0 272727.0
27545 66608 43840 57083 51949 1002.0 11596.0 3957.0 5740.0 4081.0
273401.0 2

27569 66548 43891 57070 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273438.0 272727.0
27566 66548 43923 57074 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273471.0 272727.0
27545 66525 43872 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273370.0 272727.0
27546 66548 43894 57114 51911 1002.0 11596.0 3957.0 5740.0 4081.0
273389.0 272727.0
27549 66478 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273295.0 272727.0
27546 66548 43900 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273422.0 272727.0
27573 66657 43833 57068 51990 1002.0 11596.0 3957.0 5740.0 4081.0
273497.0 272727.0
27545 66548 43840 57025 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273318.0 272727.0
27545 66549 43840 57068 51991 1002.0 11596.0 3957.0 5740.0 4081.0
273369.0 272727.0
27570 66575 43855 57095 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273455.0 272727.0
27547 66548 43838 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273361.0 272727.0
27556 66548 43853 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273385.0 2

27556 66548 43841 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273373.0 272727.0
27545 66587 43878 57097 52064 1002.0 11596.0 3957.0 5740.0 4081.0
273547.0 272727.0
27547 66592 43836 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273403.0 272727.0
27557 66548 43840 57084 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273389.0 272727.0
27547 66541 43840 57052 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273340.0 272727.0
27545 66548 43840 57086 52046 1002.0 11596.0 3957.0 5740.0 4081.0
273441.0 272727.0
27549 66548 43887 57081 51704 1002.0 11596.0 3957.0 5740.0 4081.0
273145.0 272727.0
27545 66548 43863 57111 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273427.0 272727.0
27556 66562 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273386.0 272727.0
27575 66493 43840 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273336.0 272727.0
27545 66548 43841 57072 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273366.0 272727.0
27552 66548 43873 57085 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273418.0 2

27553 66548 43895 57068 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273424.0 272727.0
27582 66548 43804 57068 52005 1002.0 11596.0 3957.0 5740.0 4081.0
273383.0 272727.0
27545 66575 43859 57049 51984 1002.0 11596.0 3957.0 5740.0 4081.0
273388.0 272727.0
